# Assignment 6

## Part 1: Preparing data

(1) Download the data

(2) Unpack the data

In [1]:
import requests
import gzip
import os
import re
import numpy as np

def download(url, to):
    response = requests.get(url)
    directory = ''.join(re.split('[/\\\]', to)[:-1])
    os.makedirs(directory, exist_ok=True)
    with open(to, "wb") as f:
        f.write(response.content)
        return to

def gz_unzip(file, out=None):
    if out is None:
        out = file.rstrip('.gz')
    
    with gzip.open(file, 'rb') as f_in:
        with open(out, 'wb') as f_out:
            f_out.write(f_in.read())
            return out

In [2]:
zip_file = download('https://snap.stanford.edu/data/facebook_combined.txt.gz', to='temp/facebook.txt.gz')
print(zip_file)
txt_file = gz_unzip(zip_file)
print(txt_file)

temp/facebook.txt.gz
temp/facebook.txt


(3) Import the data as an undirected graph in `networkx`

In [3]:
import networkx as nx

graph = nx.read_edgelist(txt_file, nodetype=int)
nx.info(graph)

'Name: \nType: Graph\nNumber of nodes: 4039\nNumber of edges: 88234\nAverage degree:  43.6910'

## Part 2: Analyse the data

Find:

(1) The number of nodes in the network

In [4]:
nx.number_of_nodes(graph)

4039

(2) The number of edges in the network

In [5]:
nx.number_of_edges(graph)

88234

(3) The average degree in the network

In [6]:
def average_degree(graph):
    return sum(dict(nx.degree(graph)).values()) / nx.number_of_nodes(graph)

average_degree(graph)

43.69101262688784

(4) A visualisation of the network inside your notebook

In [ ]:
nx.draw(graph)

## Part 3: Find the most popular people

(1) We're naturally interested in who has the most friends, so we want to extract top 10. That is, the 10 most connected people.

In [17]:
def top(graph, n = 10):
    dictionary = dict(nx.degree(graph))
    dict_sorted = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    return [node[0] for node in dict_sorted][:n]

top_connected = top(graph, 10)
top_connected

[107, 1684, 1912, 3437, 0, 2543, 2347, 1888, 1800, 1663]

(2) Finding top n using the PageRank algoritm.

In [5]:
from tqdm import tqdm

GLOBAL_RANKS = np.ones(nx.number_of_nodes(graph))

def page_rank_it(node, n):
    return GLOBAL_RANKS[node] / n

def page_rank(graph, iterations = 10):
    
    d = 0.08
    n = nx.number_of_nodes(graph)
                        
    for _ in tqdm(enumerate(range(iterations))):
    
        for node in nx.nodes(graph):
            edges = graph.edges(node)
            sum_ = 0
            for edge in edges:
                sum_ += page_rank_it(edge[1], n)
            rank = (sum_ * d) + (1 - d) / n
            GLOBAL_RANKS[node] = rank
        
page_rank(graph)

10it [00:08,  1.24it/s]


In [16]:
def top_page_rank(n = 10):
    return sorted(graph.nodes, key=lambda node: GLOBAL_RANKS[node], reverse=True)[:n]

top_page_rank = top_page_rank(10)
top_page_rank

[107, 1684, 1912, 3437, 0, 2543, 2347, 1888, 1800, 1663]

In [18]:
top_connected == top_page_rank

True